In [2]:
from google.colab import files
file = files.upload()

Saving train_data.csv to train_data.csv


In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import os
import numpy as np
import pandas as pd

train_data = pd.read_csv("train_data.csv")
train_data = train_data.iloc[:, 1:]
# test_data = pd.read_csv("./data/test_data.csv")
# test_data = test_data.iloc[:, 1:]

train_data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,time,event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1


In [4]:
X = train_data.iloc[:, :18]
y = train_data.iloc[:, 18]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [40]:
model = Sequential([Input(shape=(18, )),

                    # Dense(32),
                    # Activation('relu'),

                    # Dense(64),                    
                    # Activation('relu'),
                    
                    # Dense(128),
                    # Activation('relu'),
                    
                    # Dense(256),
                    # Activation('relu'),
                    # Dropout(0.5),
                    
                    # Dense(512),
                    # Activation('relu'),
                    # Dropout(0.5),
                    
                    Dense(10),
                    Activation('relu'),
                    Dropout(0.5),
                    
                    Dense(1, activation='linear')
])

# model.compile(optimizer=Adam(lr=0.01, decay=0.0001),
model.compile(optimizer=SGD(lr=0.01, momentum=0.5),
              loss='mae',
              metrics=['mse'])

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp = ModelCheckpoint(filepath=checkpoint_path,
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=1)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, verbose=1, callbacks=[cp, es])  # starts training

Epoch 1/1000
 1/17 [>.............................] - ETA: 0s - loss: 4.8690 - mse: 112.8626
Epoch 00001: val_loss improved from inf to 3.28723, saving model to training_1/cp.ckpt
17/17 [==============================] - 0s 10ms/step - loss: 5.0288 - mse: 432.3743 - val_loss: 3.2872 - val_mse: 67.0743
Epoch 2/1000
 1/17 [>.............................] - ETA: 0s - loss: 3.6000 - mse: 70.9573
Epoch 00002: val_loss improved from 3.28723 to 3.24861, saving model to training_1/cp.ckpt
17/17 [==============================] - 0s 4ms/step - loss: 4.9447 - mse: 428.6638 - val_loss: 3.2486 - val_mse: 66.2826
Epoch 3/1000
 1/17 [>.............................] - ETA: 0s - loss: 7.7847 - mse: 521.4067
Epoch 00003: val_loss improved from 3.24861 to 3.22589, saving model to training_1/cp.ckpt
17/17 [==============================] - 0s 4ms/step - loss: 4.9233 - mse: 431.0232 - val_loss: 3.2259 - val_mse: 65.8074
Epoch 4/1000
 1/17 [>.............................] - ETA: 0s - loss: 7.0558 - mse: 29

In [41]:
model.load_weights(checkpoint_path)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 10)                190       
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [42]:
from google.colab import files
file = files.upload()

test_data = pd.read_csv("test_data.csv")
test_data = test_data.iloc[:, 1:]

Saving test_data.csv to test_data (1).csv


In [43]:
test_data_0 = test_data.copy()
test_data_1 = test_data.copy()

In [44]:
test_data_0['Action'] = 0
test_data_1['Action'] = 1

test_data = pd.concat([test_data_0, test_data_1], axis=0)
test_data = test_data.reset_index(drop=True)
test_data

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action
0,0.00,0.10,0.40,0.03,0.46,0.31,0.50,0.15,0.55,0.06,0.68,0.51,0.82,0.57,0.28,0.77,0.74,0
1,0.52,0.75,0.56,0.78,0.28,0.05,0.52,0.45,0.26,0.05,0.43,0.21,0.88,0.97,0.18,0.05,0.97,0
2,0.45,0.78,0.47,0.46,0.80,0.51,0.23,0.28,0.88,0.58,0.47,0.41,0.87,0.79,0.73,0.55,0.08,0
3,0.58,0.29,0.68,0.97,0.20,0.43,0.60,0.68,0.37,0.63,0.83,0.67,0.83,0.18,0.49,0.75,0.54,0
4,0.38,0.02,0.58,0.51,0.58,0.15,0.45,0.27,0.56,0.09,0.98,0.50,0.39,0.88,0.81,0.18,0.17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0.31,0.55,0.91,0.37,0.24,0.57,0.33,0.81,0.30,0.09,0.22,0.54,0.88,0.49,0.10,0.53,0.47,1
568,0.39,0.52,0.19,0.36,0.50,0.08,0.08,0.50,0.42,0.52,0.86,0.13,0.80,0.85,0.98,0.27,0.95,1
569,0.98,0.87,0.45,0.99,0.94,0.81,0.31,0.00,0.94,0.73,0.70,0.91,0.45,0.34,0.86,0.84,0.58,1
570,0.43,0.69,0.04,0.59,0.44,0.57,0.66,0.13,0.17,0.40,0.91,0.25,0.25,0.22,0.42,0.79,0.61,1


In [45]:
pred = model.predict(test_data)

In [46]:
results = []
num_data = int(len(test_data) / 2) - 1
print(num_data)

for idx in range(len(pred)):
    if idx > num_data:
        break
    else:
        if pred[idx] > pred[idx + num_data]:
            results.append(0)
        else:
            results.append(1)

result = pd.DataFrame(zip(list(range(287)), results), columns=['Title', 'action'])
# result.to_csv('./result/lgb_result_change_thres.csv', index=False)
result

285


,Title,action
0,0,1
1,1,1
2,2,0
3,3,1
4,4,1
...,...,...
281,281,0
282,282,0
283,283,0
284,284,1


In [47]:
result.to_csv('keras_regression_2.csv', index=False)
files.download('keras_regression_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>